In [29]:
import pandas as pd 
# from sodapy import Socrata  # API for data

In [30]:
# Use the API to get raw data 
# client = Socrata("data.cincinnati-oh.gov", None)
# crash_raw = pd.DataFrame.from_records(
#                 client.get('rvmt-pkmq', limit=1000000)
#             )

# Using the API is faster than using pd.read_json

In [31]:
# Get the raw data 
raw = pd.read_csv("Traffic_Crash_Reports__CPD_.csv")

C:\Users\Ern\AppData\Local\Temp\ipykernel_17800\1786469094.py:2: DtypeWarning: Columns (8,27) have mixed types. Specify dtype option on import or set low_memory=False.
  raw = pd.read_csv("Traffic_Crash_Reports__CPD_.csv")


#  Overview of the Raw Data

In [32]:
raw.shape

(328558, 28)

In [33]:
raw.columns

Index(['ADDRESS_X', 'LATITUDE_X', 'LONGITUDE_X', 'AGE',
       'COMMUNITY_COUNCIL_NEIGHBORHOOD', 'CPD_NEIGHBORHOOD',
       'SNA_NEIGHBORHOOD', 'CRASHDATE', 'CRASHLOCATION', 'CRASHSEVERITY',
       'CRASHSEVERITYID', 'DATECRASHREPORTED', 'DAYOFWEEK', 'GENDER',
       'INJURIES', 'INSTANCEID', 'LIGHTCONDITIONSPRIMARY', 'LOCALREPORTNO',
       'MANNEROFCRASH', 'ROADCONDITIONSPRIMARY', 'ROADCONTOUR', 'ROADSURFACE',
       'ROADCLASS', 'ROADCLASSDESC', 'UNITTYPE', 'TYPEOFPERSON', 'WEATHER',
       'ZIP'],
      dtype='object')

In [34]:
raw.iloc[:,0:10].head()  # What does the table look like?

,ADDRESS_X,LATITUDE_X,LONGITUDE_X,AGE,COMMUNITY_COUNCIL_NEIGHBORHOOD,CPD_NEIGHBORHOOD,SNA_NEIGHBORHOOD,CRASHDATE,CRASHLOCATION,CRASHSEVERITY
0,S I75 AT 2-7 MM,39.125153,-84.537191,NaN,WEST END,QUEENSGATE,QUEENSGATE,03/30/2023 12:57:00 AM,NaN,5 - PROPERTY DAMAGE ONLY
1,2XX E LIBERTY ST,39.113294,-84.513194,32.0,MOUNT AUBURN,MOUNT AUBURN,OVER-THE-RHINE,03/29/2023 11:45:00 PM,NaN,5 - PROPERTY DAMAGE ONLY
2,2XX E LIBERTY ST,39.112334,-84.513164,24.0,MOUNT AUBURN,MOUNT AUBURN,OVER-THE-RHINE,03/29/2023 11:45:00 PM,NaN,5 - PROPERTY DAMAGE ONLY
3,6XX CENTRAL AV,39.101560,-84.519050,45.0,WEST END,QUEENSGATE,WEST END,03/29/2023 10:29:00 PM,NaN,5 - PROPERTY DAMAGE ONLY
4,6XX CENTRAL AV,39.102170,-84.519620,38.0,WEST END,QUEENSGATE,WEST END,03/29/2023 10:29:00 PM,NaN,5 - PROPERTY DAMAGE ONLY


We note that there are several columns that can be split up into the "quantitative measure" and description. These columns are: 
- `CRASHSEVERITY`
- `GENDER`
- `INJURIES`
- `LIGHTCONDITIONSPRIMARY`
- `MANNEROFCRASH`
- `ROADCONDITIONSPRIMARY`
- `ROADSURFACE`
- `UNITTYPE`
- `TYPEOFPERSON`
- `WEATHER`

In [35]:
data.INJURIES.unique()

array([nan, '5 - NO APPARENTY INJURY', '3 - SUSPECTED MINOR INJURY',
       '4 - POSSIBLE INJURY', '2 - SUSPECTED SERIOUS INJURY', '1 - FATAL',
       '1 - NO INJURY / NONE REPORTED', '2 - POSSIBLE',
       '3 - NON-INCAPACITATING', '4 - INCAPACITATING', '5 - FATAL'],
      dtype=object)

In [36]:
raw.dtypes  # Columns and data types

ADDRESS_X                          object
LATITUDE_X                        float64
LONGITUDE_X                       float64
AGE                               float64
COMMUNITY_COUNCIL_NEIGHBORHOOD     object
CPD_NEIGHBORHOOD                   object
SNA_NEIGHBORHOOD                   object
CRASHDATE                          object
CRASHLOCATION                      object
CRASHSEVERITY                      object
CRASHSEVERITYID                   float64
DATECRASHREPORTED                  object
DAYOFWEEK                          object
GENDER                             object
INJURIES                           object
INSTANCEID                         object
LIGHTCONDITIONSPRIMARY             object
LOCALREPORTNO                       int64
MANNEROFCRASH                      object
ROADCONDITIONSPRIMARY              object
ROADCONTOUR                        object
ROADSURFACE                        object
ROADCLASS                         float64
ROADCLASSDESC                     

In addition, converting the data types of some of the fields would be useful for analysis. These fields are:
- `CRASHDATE` `object` to datetime
- `LOCALREPORTNO` `int64` to `str`
- `DATECRASHREPORED` `object` to datetime
- `CRASHSEVERITYID` `float64` to `str`

Finally, we notice that `ZIP` is string but has decimal points. We will need to remove those.

# Reformatting Data

In [37]:
data = raw 

##  Converting data types

In [38]:
# Strings to dates
data.loc[:, 'CRASHDATE'] = pd.to_datetime(data.CRASHDATE)
data.loc[:, 'DATECRASHREPORTED'] = pd.to_datetime(data.DATECRASHREPORTED)

# Numbers to strings
data.loc[:, 'LOCALREPORTNO'] = data.LOCALREPORTNO.apply(str )
data.loc[:, 'CRASHSEVERITYID'] = data.CRASHSEVERITYID.apply(str)

C:\Users\Ern\AppData\Local\Temp\ipykernel_17800\1858008429.py:2: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[:, 'CRASHDATE'] = pd.to_datetime(data.CRASHDATE)
C:\Users\Ern\AppData\Local\Temp\ipykernel_17800\1858008429.py:3: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  data.loc[:, 'DATECRASHREPORTED'] = pd.to_datetime(data.DATECRASHREPORTED)


In [39]:
# Stripping the trailing zeros from ZIP 
data.loc[:,'ZIP'] = data.ZIP.astype(str)
data.loc[:,'ZIP'] = data.ZIP.str[:5]

## Splitting fields

In [40]:
data.WEATHER.unique()

array(['1 - CLEAR', '2 - CLOUDY', '99 - OTHER/UNKNOWN', '4 - RAIN',
       '6 - SNOW', '9 - FREEZING RAIN OR FREEZING DRIZZLE',
       '7 - SEVERE CROSSWINDS', '5 - SLEET, HAIL', '3 - FOG, SMOG, SMOKE',
       '8 - BLOWING SAND, SOIL, DIRT, SNOW', '9 - OTHER/UNKNOWN',
       '5 - SLEET,HAIL', nan], dtype=object)

In [41]:
# Separate scores from descriptions
data[['CRASHSEVERITY','CRASHSEVERITYDESCR']] = data.CRASHSEVERITY.str.split(pat=' - ', expand=True)
data.CRASHSEVERITY = data.CRASHSEVERITY.astype('Int64')

data[['ROADCONTOUR','ROADCONTOURDESCR']] = data.ROADCONTOUR.str.split(pat=' - ', expand=True)
data.ROADCONTOUR = data.ROADCONTOUR.astype('Int64')

data[['MANNEROFCRASH', 'MANNEROFCRASHDESCR']] = data.MANNEROFCRASH.str.split(pat=' - ', expand=True)
data.MANNEROFCRASH = data.MANNEROFCRASH.astype('Int64')

data[['TYPEOFPERSON','TYPEOFPERSONDESCR']] = data.TYPEOFPERSON.str.split(pat=' - ', expand=True)

Some fields are more complicated than a simple string split. These fields are:
- `GENDER`: Some values have a code and a description, others just a description. 
- `INJURIES`: There seem to be two different measurement systems, one ascending in severity and one descending in severity.
- `LIGHTCONDITIONSPRIMARY`: There are multiple values, which can conflict, per score.
- `ROADCONDITIONSPRIMARY`: There are multiple values per score.
- `ROADSURFACE`: There are multiple values per score and the same value mapped to differing scores.
- `UNITTYPE`: There are multiple values per score.
- `WEATHER`: There are multiple values per score.

Let us examine these fields in greater detail.

### Gender

In [42]:
data.GENDER.unique()

array([nan, 'F - FEMALE', 'M - MALE', 'U - UNKNOWN', 'FEMALE', 'MALE'],
      dtype=object)

The space the description strings take up is negligible, so let us remove the codes from the field to make the values consistent.

In [45]:
data['GENDER'] = data.GENDER.str.replace('F - ', '')
data['GENDER'] = data.GENDER.str.replace('M - ', '')
data['GENDER'] = data.GENDER.str.replace('U - ', '')
data.GENDER.unique()

array([nan, 'FEMALE', 'MALE', 'UNKNOWN'], dtype=object)

# Other

In [16]:
data.dtypes

ADDRESS_X                          object
LATITUDE_X                        float64
LONGITUDE_X                       float64
AGE                               float64
COMMUNITY_COUNCIL_NEIGHBORHOOD     object
CPD_NEIGHBORHOOD                   object
SNA_NEIGHBORHOOD                   object
CRASHDATE                          object
CRASHLOCATION                      object
CRASHSEVERITY                      object
CRASHSEVERITYID                   float64
DATECRASHREPORTED                  object
DAYOFWEEK                          object
GENDER                             object
INJURIES                           object
INSTANCEID                         object
LIGHTCONDITIONSPRIMARY             object
LOCALREPORTNO                       int64
MANNEROFCRASH                      object
ROADCONDITIONSPRIMARY              object
ROADCONTOUR                        object
ROADSURFACE                        object
ROADCLASS                         float64
ROADCLASSDESC                     

In [99]:
data.shape

(324723, 30)

In [17]:
len(data.instance_id.unique())

AttributeError: 'DataFrame' object has no attribute 'instance_id'

Since the size of the table is larger than the number of unique IDs, we know that one accident may be captured in several rows.

In [11]:
raw.describe()

,LATITUDE_X,LONGITUDE_X,AGE,CRASHSEVERITYID,LOCALREPORTNO,ROADCLASS
count,3.285290e+05,3.285270e+05,287961.000000,328551.000000,3.285580e+05,203133.000000
mean,2.633354e+03,-1.199095e+04,37.465070,82733.375893,1.846722e+08,2.825745
std,1.469781e+06,6.819152e+06,16.585971,99293.182335,7.679149e+08,1.484126
min,-8.449023e+01,-3.908552e+09,0.000000,1.000000,1.400000e+01,0.000000
25%,3.911740e+01,-8.454953e+01,24.000000,3.000000,1.550171e+08,2.000000
50%,3.913663e+01,-8.451502e+01,34.000000,3.000000,1.850029e+08,2.000000
75%,3.916096e+01,-8.448473e+01,49.000000,201905.000000,2.050094e+08,4.000000
max,8.424329e+08,1.000000e-03,933.000000,201905.000000,2.201140e+11,10.000000


In [12]:
data.address_x    # Will want to separate block # from street.

AttributeError: 'DataFrame' object has no attribute 'address_x'

In [13]:
data.head()

,ADDRESS_X,LATITUDE_X,LONGITUDE_X,AGE,COMMUNITY_COUNCIL_NEIGHBORHOOD,CPD_NEIGHBORHOOD,SNA_NEIGHBORHOOD,CRASHDATE,CRASHLOCATION,CRASHSEVERITY,...,MANNEROFCRASH,ROADCONDITIONSPRIMARY,ROADCONTOUR,ROADSURFACE,ROADCLASS,ROADCLASSDESC,UNITTYPE,TYPEOFPERSON,WEATHER,ZIP
0,S I75 AT 2-7 MM,39.125153,-84.537191,NaN,WEST END,QUEENSGATE,QUEENSGATE,03/30/2023 12:57:00 AM,NaN,5 - PROPERTY DAMAGE ONLY,...,"7 - SIDESWIPE, SAME DIRECTION",01 - DRY,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",NaN,NaN,NaN,NaN,1 - CLEAR,45223.0
1,2XX E LIBERTY ST,39.113294,-84.513194,32.0,MOUNT AUBURN,MOUNT AUBURN,OVER-THE-RHINE,03/29/2023 11:45:00 PM,NaN,5 - PROPERTY DAMAGE ONLY,...,6 - ANGLE,01 - DRY,2 - STRAIGHT GRADE,"2 - BLACKTOP, BITUMINOUS, ASPHALT",NaN,NaN,19 - BUS (16+ PASSENGERS),D - DRIVER,1 - CLEAR,45202.0
2,2XX E LIBERTY ST,39.112334,-84.513164,24.0,MOUNT AUBURN,MOUNT AUBURN,OVER-THE-RHINE,03/29/2023 11:45:00 PM,NaN,5 - PROPERTY DAMAGE ONLY,...,6 - ANGLE,01 - DRY,2 - STRAIGHT GRADE,"2 - BLACKTOP, BITUMINOUS, ASPHALT",NaN,NaN,01 - PASSENGER CAR,O - OCCUPANT,1 - CLEAR,45202.0
3,6XX CENTRAL AV,39.101560,-84.519050,45.0,WEST END,QUEENSGATE,WEST END,03/29/2023 10:29:00 PM,NaN,5 - PROPERTY DAMAGE ONLY,...,2 - REAR-END,01 - DRY,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",6.0,Ramps,01 - PASSENGER CAR,D - DRIVER,1 - CLEAR,45203.0
4,6XX CENTRAL AV,39.102170,-84.519620,38.0,WEST END,QUEENSGATE,WEST END,03/29/2023 10:29:00 PM,NaN,5 - PROPERTY DAMAGE ONLY,...,2 - REAR-END,01 - DRY,1 - STRAIGHT LEVEL,"2 - BLACKTOP, BITUMINOUS, ASPHALT",6.0,Ramps,03 - SPORT UTILITY VEHICLE,D - DRIVER,1 - CLEAR,45203.0
